#### 使用Scipy求解SVM

In [3]:
from scipy.optimize import minimize
import numpy as np
class SVM:
    def __init__(self,C=0.001):
        self.c=C
        self.relaxation_type='eq' if C==0 else 'ineq'
        
    def fit(self,X,y):
        y=2*y-1
        m,n=X.shape
        unknow_vector=np.ones(m+n+1)/(m+n+1)
        
        self.solver=minimize(type(self).minimize_object,unknow_vector,args=(X,y,self.c),
              constraints=({'type': 'ineq', 'args': (X,y),
                            'fun':lambda unknow_vector,X,y:type(self).linear_onstraint(unknow_vector,X,y)},
                           {'type':self.relaxation_type, 'args': (y,),'fun':lambda unknow_vector,y:type(self).relaxation_constraint(unknow_vector,y)}                         
             ))
        self.coef_=self.solver.x[:n]
        self.intercept_=b=self.solver.x[n]
        self.beta_=self.solver.x[n+1:]
        
        self.support_value=y*(np.dot(X,self.coef_.T)+self.intercept_)+self.beta_
        self.support_=np.where(self.support_value<=1.01)
        self.support_vectors_=X[self.support_]
           
        return self
    
    def predict(self,X):
        pred=X@self.coef_+self.intercept_
        return (np.sign(pred)+1)/2
        
    
    
    def minimize_object(unknow_vector,X,y,c):
        m,n=X.shape
        w=unknow_vector[:n]  
        b=unknow_vector[n]   
        beta=unknow_vector[n+1:]
        object_value=0.5*np.sum(w**2)+c*np.sum(beta)
        return object_value

    def linear_onstraint(unknow_vector,X,y):
        m,n=X.shape
        w=unknow_vector[:n]
        b=unknow_vector[n]
        beta=unknow_vector[n+1:]
        return y*(X@w.T+b)+beta-1

    def relaxation_constraint(unknow_vector,y):
        n=len(y)
        beta=unknow_vector[n+1:]    
        return beta

In [7]:
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X,y=load_breast_cancer().data,load_breast_cancer().target
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)
my_svm=SVM(C=1).fit(X_train,y_train)
pred=my_svm.predict(X_test)
pred

array([1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1.,
       1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1.,
       1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1.,
       1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1.,
       0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
       1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1.,
       1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.,
       1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
       1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
       0.])

In [8]:
accuracy_score(y_test,pred)

0.9298245614035088

In [9]:
my_svm.support_vectors_

array([[1.458e+01, 2.153e+01, 9.741e+01, 6.448e+02, 1.054e-01, 1.868e-01,
        1.425e-01, 8.783e-02, 2.252e-01, 6.924e-02, 2.545e-01, 9.832e-01,
        2.110e+00, 2.105e+01, 4.452e-03, 3.055e-02, 2.681e-02, 1.352e-02,
        1.454e-02, 3.711e-03, 1.762e+01, 3.321e+01, 1.224e+02, 8.969e+02,
        1.525e-01, 6.643e-01, 5.539e-01, 2.701e-01, 4.264e-01, 1.275e-01],
       [1.585e+01, 2.395e+01, 1.037e+02, 7.827e+02, 8.401e-02, 1.002e-01,
        9.938e-02, 5.364e-02, 1.847e-01, 5.338e-02, 4.033e-01, 1.078e+00,
        2.903e+00, 3.658e+01, 9.769e-03, 3.126e-02, 5.051e-02, 1.992e-02,
        2.981e-02, 3.002e-03, 1.684e+01, 2.766e+01, 1.120e+02, 8.765e+02,
        1.131e-01, 1.924e-01, 2.322e-01, 1.119e-01, 2.809e-01, 6.287e-02],
       [1.315e+01, 1.534e+01, 8.531e+01, 5.389e+02, 9.384e-02, 8.498e-02,
        9.293e-02, 3.483e-02, 1.822e-01, 6.207e-02, 2.710e-01, 7.927e-01,
        1.819e+00, 2.279e+01, 8.584e-03, 2.017e-02, 3.047e-02, 9.536e-03,
        2.769e-02, 3.479e-03, 1.477e